In [1]:
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime
from itertools import compress
import time
import os

In [3]:
options = {'driver':r'C:\Users\likith\Desktop\QGIS\project\chromedriver.exe',
           'output':'ALL_Data',
           'sdate':'01-01-2019'
          }

In [6]:
global url,option,browser
url = 'https://app.cpcbccr.com/AQI_India/#' 
option=webdriver.ChromeOptions()
browser = webdriver.Chrome(executable_path="{}".format(options['driver']), chrome_options=option)
browser.get(url)
time.sleep(5)

<ipython-input-6-029efec1e6d7>:4: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(executable_path="{}".format(options['driver']), chrome_options=option)


In [7]:
def get_states():
    st = browser.page_source
    soup = BeautifulSoup(st, 'html.parser')
    table = soup.find("select",{"id":"states"})
    states  = []
    for i in table.find_all("option"):
        states.append(i.text)
    return states[1:]

In [8]:
def get_cities(states,state):
    path = '//*[@id="states"]/option['+str(states.index(state)+2)+']'
    browser.find_element_by_xpath(path).click()
    time.sleep(5)
    st = browser.page_source
    soup = BeautifulSoup(st, 'html.parser')
    table2 = soup.find("select",{"id":"cities"})
    cities  = []
    for i in table2.find_all("option"):
        cities.append(i.text)
    return cities[1:]

In [9]:
def get_stations(cities,city):
    path = '//*[@id="cities"]/option['+str(cities.index(city)+2)+']'
    browser.find_element_by_xpath(path).click()
    time.sleep(5)
    st = browser.page_source
    soup = BeautifulSoup(st, 'html.parser')
    table2 = soup.find("select",{"id":"stations"})
    stations  = []
    for i in table2.find_all("option"):
        stations.append(i.text)
    return stations[1:]

In [28]:
def extract_data(stations,date):
    date = date
    stations=stations
    over_data = []
    path = '//*[@id="time"]'
    browser.find_element_by_xpath(path).click()
    browser.find_element_by_xpath(path).clear()
    time.sleep(0.5)
    browser.find_element_by_xpath(path).send_keys('0:00')
    time.sleep(0.5)
    browser.find_element_by_xpath('/html/body').click()
    path = '//*[@id="date"]/input'
    browser.find_element_by_xpath(path).click()
    browser.find_element_by_xpath(path).clear()
    time.sleep(2)
    browser.find_element_by_xpath(path).send_keys(date)
    time.sleep(2)
    browser.find_element_by_xpath('/html/body').click()
    for i in stations:
        time.sleep(3)
        data =[]
        s = [i]
        path = '//*[@id="stations"]/option['+str(stations.index(i)+2)+']'
        browser.find_element_by_xpath(path).click()
        time.sleep(2)
        st = browser.page_source
        soup = BeautifulSoup(st, 'html.parser')
        tab =soup.find_all('td',{"class":"element-name"})
        parameters  = []
        for i in tab:
            parameters.append(i.text)
        req = ['PM2.5','PM10']
        #req = ['PM2.5']

        tab =soup.find_all('div',{"aria-label":"A tabular representation of the data in the chart."})
        try:
            count = 0
            date = []
            pm = []
            x =tab[parameters.index(req[0])].find_all('td')
            y =tab[parameters.index(req[1])].find_all('td')
            for j in x:
                if count%2 == 1:
                    pm.append(j.text)
                else:
                    date.append(j.text)
                count = count+1
            data.append(list(s))
            data.append(list(pm))
            data.append(list(date))
            count = 0
            date = []
            pm = []
            for j in y:
                if count%2 == 1:
                    pm.append(j.text)
                else:
                    date.append(j.text)
                count = count+1
            data.append(list(pm))
            data.append(list(date))
        except:
            pass
        if len(data) >0:
            over_data.append(data)
    return over_data

In [ ]:
states = get_states()
for i in states:
    cities = get_cities(states,i)
    for j in cities:
        stations = get_stations(cities,j)      
        for k in stations:
            

In [12]:
cities = []
states = get_states()
print(states)
inp = input('Enter the state name here : ')
if inp != 'All':
    cities = get_cities(states,inp)
else:
    for i in states:
        print(get_cities(states,i))

['Andhra Pradesh', 'Arunachal Pradesh', 'Assam', 'Bihar', 'Chandigarh', 'Chhattisgarh', 'Delhi', 'Gujarat', 'Haryana', 'Jammu and Kashmir', 'Jharkhand', 'Karnataka', 'Kerala', 'Madhya Pradesh', 'Maharashtra', 'Meghalaya ', 'Mizoram', 'Nagaland', 'Odisha', 'Puducherry', 'Punjab', 'Rajasthan', 'Tamil Nadu', 'Telangana', 'Tripura', 'Uttar Pradesh', 'West Bengal']
Enter the state name here : All
['Amaravati', 'Rajamahendravaram', 'Tirupati', 'Visakhapatnam']
['Naharlagun']
['Guwahati']
['Bettiah', 'Bihar Sharif', 'Darbhanga', 'Gaya', 'Hajipur', 'Motihari', 'Muzaffarpur', 'Patna']
['Chandigarh']
['Bilaspur']
['Delhi']
['Ahmedabad', 'Ankleshwar', 'Gandhinagar', 'Nandesari', 'Vapi', 'Vatva']
['Ambala ', 'Bahadurgarh ', 'Ballabgarh', 'Bhiwani', 'Charkhi Dadri', 'Dharuhera', 'Faridabad', 'Fatehabad ', 'Gurugram', 'Hisar', 'Jind', 'Kaithal', 'Karnal', 'Kurukshetra ', 'Mandikhera', 'Manesar', 'Narnaul', 'Palwal ', 'Panchkula', 'Panipat', 'Rohtak', 'Sirsa', 'Sonipat', 'Yamunanagar']
['Srinagar']
[

In [10]:
print(cities)
stations = []
inp = input('Enter the city name here :   ')
stations = get_stations(cities,inp)

['Bagalkot', 'Bengaluru', 'Bidar', 'Chamarajanagar', 'Chikkaballapur', 'Chikkamagaluru', 'Davanagere', 'Gadag', 'Hassan', 'Hubballi', 'Kalaburagi', 'Kolar', 'Koppal', 'Madikeri', 'Mangalore', 'Mysuru', 'Raichur', 'Ramanagara', 'Shivamogga', 'Udupi', 'Vijayapura', 'Yadgir']
Enter the city name here :   Bengaluru


In [11]:
stations

['BTM Layout, Bengaluru - CPCB',
 'BWSSB Kadabesanahalli, Bengaluru - CPCB',
 'Bapuji Nagar, Bengaluru - KSPCB',
 'City Railway Station, Bengaluru - KSPCB',
 'Hebbal, Bengaluru - KSPCB',
 'Hombegowda Nagar, Bengaluru - KSPCB',
 'Jayanagar 5th Block, Bengaluru - KSPCB',
 'Peenya, Bengaluru - CPCB',
 'Sanegurava Halli, Bengaluru - KSPCB',
 'Silk Board, Bengaluru - KSPCB']

In [21]:
def create_dataframe(over_data):
    stati = []
    p = []
    d = []
    p2 = []
    for i in over_data:
        for j in range(len(i[2])):
            stati.append(i[0][0])
        for k in range(len(i[1])):
            p.append(i[1][k])
            p2.append(i[3][k])
            if i[2][k].split(" ")[-1] == 'AM' and int(i[2][k].split(" ")[-2].split(':')[0]) != 12:
                h=int(i[2][k].split(" ")[-2].split(':')[0])
                d.append(h)
            elif int(i[2][k].split(" ")[-2].split(':')[0]) == 12 and i[2][k].split(" ")[-1] == 'PM':
                h=int(i[2][k].split(" ")[-2].split(':')[0])
                d.append(h)
            else:
                h = int(i[2][k].split(" ")[-2].split(':')[0])
                d.append(h+12)
    return (stati,p,d,p2)

In [13]:
import datetime
d1 = datetime.date(2020, 1, 1)
d2 = datetime.date(2020, 12, 31)
days = [d1 + datetime.timedelta(days=x) for x in range((d2-d1).days + 1)]
req_dates = []
for day in days:
    req_dates.append(day.strftime('%d/%m/%Y'))

In [14]:
req_dates[0]

'01/01/2020'

In [ ]:
import pandas as pd
df = pd.DataFrame(columns = ['station','time','PM2.5'])
states = get_states()
for i in states:
    cities = get_cities(states,i)
    for j in cities:
        stations = get_stations(cities,j)
        for d in req_dates:
            dta = extract_data(stations,d)
            stati,p,d,p2 = create_dataframe(dta)
            for k in range(len(stati)):
                df = df.append({'State':i,'City':j,'station':stati[k] , 'time' : d[k] , 'PM2.5':p[k],'PM10':p2[k],'date' : d},ignore_index=True)

# #if req_dates[0:3].index(i) == 0:
# stati,p,d = create_dataframe(dta)
# for j in range(len(stati)):
#     df = df.append({'station':stati[j] , 'time' : d[j] , 'PM2.5':p[j]},ignore_index=True)
# #         df['station'] = stati
# #         df['PM2.5']=p
# #         df['time'] = d
# #     else:
# #         stati,p,d = create_dataframe(dta)
# #         df['station'].append(stati)
# #         df['PM2.5'].append(p)
# #         df['time'].append(d)

In [37]:
df.tail(100)

,station,time,PM2.5,Cities,PM10,State,date
164,"Tirumala, Tirupati - APPCB",21,48,Tirupati,42,Andhra Pradesh,02/01/2020
165,"Tirumala, Tirupati - APPCB",22,41,Tirupati,36,Andhra Pradesh,02/01/2020
166,"Tirumala, Tirupati - APPCB",23,21,Tirupati,19,Andhra Pradesh,02/01/2020
167,"Tirumala, Tirupati - APPCB",24,15,Tirupati,17,Andhra Pradesh,02/01/2020
168,"Tirumala, Tirupati - APPCB",1,17,Tirupati,16,Andhra Pradesh,03/01/2020
...,...,...,...,...,...,...,...
259,"GVM Corporation, Visakhapatnam - APPCB",20,98,Visakhapatnam,107,Andhra Pradesh,03/01/2020
260,"GVM Corporation, Visakhapatnam - APPCB",21,114,Visakhapatnam,115,Andhra Pradesh,03/01/2020
261,"GVM Corporation, Visakhapatnam - APPCB",22,119,Visakhapatnam,111,Andhra Pradesh,03/01/2020
262,"GVM Corporation, Visakhapatnam - APPCB",23,122,Visakhapatnam,112,Andhra Pradesh,03/01/2020
